In [ ]:
!pip install transformers
!pip install editdistance

import editdistance
import pandas as pd
import transformers
import matplotlib.pyplot as plt
import torch
import torch.cuda
import sklearn
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn import preprocessing
from collections import defaultdict
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoModelForSequenceClassification

drive.mount('/content/drive')
#maltese dataset is size 160184
#italian dataset is size 509573
dataset_ita = pd.read_csv('/content/drive/MyDrive/ita')

dataset_copy_ita=dataset_ita.copy()
list_data_ita = dataset_copy_ita.values.tolist()
#list_data_ita=list_data_ita[0:160184]

In [ ]:
split_data_ita=[]

def split(corpus, spit_data):
  for sets in corpus:
    for x in sets:
      parts = x.split("\t")
      spit_data.append(parts)

split(list_data_ita,split_data_ita)

In [ ]:
# removing duplicates
def remove_duplicates(split_data):
    unique_data = list(set(tuple(sublist) for sublist in split_data))
    unique_data = [list(sublist) for sublist in unique_data]
    return unique_data

unique_data_ita = remove_duplicates(split_data_ita)

# getting the longest word for padding 
len_longest=0
for string in unique_data_ita:
    x=string[1]
    if len(x)>len_longest:
      len_longest=len(x)

In [ ]:
# splitting into words and grammar
def split_word_and_grammar(unique_data_array):
    last_elements = [subarray[-1] for subarray in unique_data_array]
    word_list = [(subarray[0],subarray[1]) for subarray in unique_data_array]
    return word_list, last_elements

word_list_ita, last_elements_ita = split_word_and_grammar(unique_data_ita)

# splitting grammar
def splitting_grammar(last_elements):
    list_of_lists = []
    for info in last_elements:
        parts = info.split(';')
        num_parts = [int(part) if part.isdigit() else part for part in parts]
        list_of_lists.append(num_parts)
    return list_of_lists

list_of_lists_ita = splitting_grammar(last_elements_ita)

print(word_list_ita[0:10])
print(last_elements_ita[0:10])

In [ ]:
# splitting data
tokeniser = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased") #,char_level=True
model = AutoModelForMaskedLM.from_pretrained("dbmdz/bert-base-italian-cased")

#tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
#model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")

from sklearn.model_selection import train_test_split
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

X = word_list_ita
y = list_of_lists_ita

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.2)

print(y_train[0:100])
print(X_test[0:100])

In [ ]:
import sklearn
from sklearn import preprocessing
from collections import defaultdict

# tokenize the training set
token_indexes = tokeniser(X_train, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_train_x = token_indexes['input_ids'].to(device).squeeze()
mask_train_x = token_indexes['attention_mask'].to(device).squeeze()

# tokenize the test set
new_tuples = [(("",) + t[1:]) for t in X_test]
token_indexes_test = tokeniser(new_tuples, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_test_x = token_indexes_test['input_ids'].to(device).squeeze()
mask_test_x = token_indexes_test['attention_mask'].to(device).squeeze()

# padding w/0 so gramm info is all the same length
def padding_grammar(Y):
    max_len = max(len(info) for info in Y)
    padded_array = [info + ['<PAD>'] * (max_len - len(info)) for info in Y]
    return padded_array
padded_morph_info_train = padding_grammar(y_train)
padded_morph_info_test = padding_grammar(y_test)

print(padded_morph_info_train[0:100])
# creating a dictionary for mapping 
morph_vocab = defaultdict(lambda: len(morph_vocab))
for i in range(len(padded_morph_info_train[0])):
    # extracting the unique values at this position
    values = set(info[i] for info in padded_morph_info_train)
    # assignign index
    for value in values:
        morph_vocab[value]

In [ ]:
# map the morphological information to indexes using the vocabulary
def get_targets_one_hot(padded_info):
  targets = [[morph_vocab[value] for value in info] for info in padded_info]
  targets = [[morph_vocab['<PAD>'] if value == 0 else value for value in info] for info in targets]
  targets=torch.tensor(targets).to(device)

  # one-hot encode the targets
  num_classes = len(morph_vocab)
  targets = torch.nn.functional.one_hot(targets, num_classes=num_classes).to(torch.float32)
  targets = torch.tensor(targets, requires_grad=True).to(device)
  return targets

targets_train = get_targets_one_hot(padded_morph_info_train)
targets_test = get_targets_one_hot(padded_morph_info_test)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.output_layer1 = torch.nn.Linear(768, 500)
        self.output_layer2 = torch.nn.Linear(500, 300)
     #   self.output_layer3 = torch.nn.Linear(300, 100)
        self.output_layer4 = torch.nn.Linear(300, 6*len(morph_vocab)) 
        self.drop = torch.nn.Dropout(p=0.2)

    def forward(self, x, mask):
        with torch.cuda.amp.autocast():
            vecs = self.bert(x, attention_mask=mask, output_hidden_states=True).hidden_states[8][:, 0, :]
            output = self.output_layer1(vecs)
            output = torch.relu(output)
            output = self.output_layer2(output)
            output = torch.relu(output)
         #   output = self.output_layer3(output)
         #   output = torch.relu(output)
            output = self.output_layer4(output)
         #   output = self.drop(output)
            output = output.view(-1, 6, len(morph_vocab))
            output = torch.sigmoid(output)
        return output

bert = transformers.BertForMaskedLM.from_pretrained("dbmdz/bert-base-italian-cased")
#bert = transformers.BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model = Model(bert)
model.to(device)


In [ ]:
def grammatical_accuracy(output, target):
    output = output.detach().cpu().numpy()
    target = target.detach().cpu().numpy()

    output = np.argmax(output, axis=1)
    target = np.argmax(target, axis=1)
    accuracy = np.mean(output == target) * 100
    return accuracy

def get_acc(matches):
    if len(matches) == 0:
        return 0
    return sum(matches)/len(matches)
#return round(100*sum(preds)/len(preds),3)

In [ ]:
batch_size=32

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

print('step', 'error', 'accuracy')
train_errors = []

accumulation_steps = 4
for step in range(1, 200+1):
    optimizer.zero_grad()
    model.train(True)
    loss_fn = torch.nn.CrossEntropyLoss()
    #.BCELoss()
    #.CrossEntropyLoss()
    #.BCEWithLogitsLoss()
   
    all_outputs = []
    all_targets = []
    for i in range(accumulation_steps):
    # print(targets_train.shape)
      output = model(indexed_train_x[i*batch_size:(i+1)*batch_size], mask_train_x[i*batch_size:(i+1)*batch_size])
      
    # MSELoss   
    # output=output.to(torch.float16)
    # targets_train=targets_train.to(torch.float16)

    # m = torch.nn.Sigmoid()
      error = loss_fn(output, targets_train[i*batch_size:(i+1)*batch_size])
      error = error/accumulation_steps
      error.backward()

      all_outputs.append(output.cpu().detach().numpy())
      all_targets.append(targets_train[i*batch_size:(i+1)*batch_size].cpu().detach().numpy())

    optimizer.step()
    model.train(False)

    train_errors.append(error.detach().tolist())

    train_accs = []
    
  #  one_hot_output = np.round(output.cpu().detach()).numpy().astype(int)
    
    for x in range(0,batch_size): # every vector in the output
      targets = targets_train[i*batch_size:(i+1)*batch_size].cpu().detach()
      targets = targets[x].argmax(dim=1).tolist()
      predicted_labels = output.cpu().detach()[x].argmax(dim=1).tolist()  
      results=0

     # print('t',targets)
     # print('p',predicted_labels)
      for vecs1 in predicted_labels: # every vector in output vector i
        for vecs2 in targets: # every vector in target vector i
            if vecs1==vecs2:
                results=results+1
          
      if results>=len(predicted_labels): #0.7*len(predicted_labels):
        train_accs.append(1)
      else:
        train_accs.append(0)  
    accuracy=get_acc(train_accs)

    if step % 10 == 0:
        print(step, train_errors[-1], accuracy)

In [ ]:
with torch.no_grad():
    print('sent', 'prediction')
    for i in range(len(indexed_test_x) // batch_size):
        outputs = model(indexed_test_x[i*batch_size:(i+1)*batch_size], mask_test_x[i*batch_size:(i+1)*batch_size])

        # reversing the dict to get the grammar from the index
        reverse_dict = {v: k for k, v in morph_vocab.items()} 

        for j, x in enumerate(outputs):
            predicted_labels = [reverse_dict[i] for i in x.argmax(dim=1).tolist()]

            # changing from char to word
            input_word = ''.join(tokeniser.decode(indexed_test_x[i*batch_size:(i+1)*batch_size][j]).split())

            print(input_word, predicted_labels)
            print("actual")
            print(y_test[i*batch_size:(i+1)*batch_size][j])

In [ ]:
(fig, ax) = plt.subplots(1, 1)
ax.set_xlabel('step')
ax.set_ylabel('$E$')
ax.plot(range(1, len(train_errors) + 1), train_errors, color='blue', linestyle='-', linewidth=3)
ax.grid()